In [ ]:
from agential.agents.react.agent import ReAct
from agential.utils.docstore import DocstoreExplorer
from agential.core.llm import LLM

import tiktoken

from langchain_community.docstore.wikipedia import Wikipedia

from agential.core.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.fever import (
    FEVER_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.triviaqa import (
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.ambignq import (
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.svamp import (
    SVAMP_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.tabmwp import (
    TABMWP_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_REACT
)
from agential.core.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_REACT
)
from agential.agents.react.prompts import (
    REACT_INSTRUCTION_HUMANEVAL, 
    REACT_INSTRUCTION_MBPP, 
    REACT_INSTRUCTION_HOTPOTQA,
    REACT_INSTRUCTION_FEVER,
    REACT_INSTRUCTION_AMBIGNQ,
    REACT_INSTRUCTION_TRIVIAQA,
    REACT_INSTRUCTION_GSM8K,
    REACT_INSTRUCTION_SVAMP,
    REACT_INSTRUCTION_TABMWP
)

import warnings
warnings.filterwarnings('ignore')

import dotenv
dotenv.load_dotenv()

llm = LLM("gpt-3.5-turbo")

In [1]:
import json

# Open and read the JSON file
with open('../data/hotpot_dev_v1_simplified.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data[:5])

[{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'answer': 'yes', 'type': 'comparison'}, {'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', 'answer': 'Chief of Protocol', 'type': 'bridge'}, {'question': 'What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?', 'answer': 'Animorphs', 'type': 'bridge'}, {'question': 'Are the Laleli Mosque and Esma Sultan Mansion located in the same neighborhood?', 'answer': 'no', 'type': 'comparison'}, {'question': 'The director of the romantic comedy "Big Stone Gap" is based in what New York city?', 'answer': 'Greenwich Village, New York City', 'type': 'bridge'}]


In [ ]:
from agential.eval.metrics.classification import EM


agent = ReAct(
    llm=llm,
    benchmark="hotpotqa",
    # kwargs.
    max_steps=8,
    max_tokens=5000,
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo"),
    docstore=DocstoreExplorer(Wikipedia()),
)


num_correct = 0 
samples = 20

for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]

    out = agent.generate(
        question=question,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT, 
        prompt=REACT_INSTRUCTION_HOTPOTQA,
        additional_keys={},
        reset=True,
    )

    is_correct = EM(out.answer, answer)
    print(question)
    print(answer, "\t\t", out.answer, "\t\t", is_correct, end="\n\n")

    num_correct += int(is_correct)
    
print(f"{num_correct}/{samples}")